# Actuarial Standards of Practice (ASOP) Q&A Machine using Retrieval Augmented Generation (RAG)
This project aims to create a Retrieval-Augmented Generation (RAG) process for actuaries to ask questions on a set of Actuarial Standards of Practice (ASOP) documents. The RAG process utilizes the power of the Large Language Model (LLM) to provide answers to questions on ASOPs.

However, RAG is not without challenges, i.e., hallucination and inaccuracy. This code allows verifiability by providing the context it used to arrive at those answers. This process enables actuaries to validate the information provided by the LLM, empowering them to make informed decisions. By combining the capabilities of LLM with verifiability, this code offers actuaries a robust tool to leverage LLM technology effectively and extract maximum value.

The current example uses either OpenAI's GPT 3.5 turbo AND a local LLM. Note that the current notebook is set up to output results from both models for comparison purposes.  
Using a local LLM can address potential data privacy or security concerns.

View license or further information about the local models used:
- Solar 10.7B Instruct: [cc-by-nc-4.0](https://huggingface.co/upstage/SOLAR-10.7B-Instruct-v1.0) (non-commercial use)
- Mistral 7B Instruct: [Apache License 2.0](https://ollama.com/library/mistral/blobs/sha256:43070e2d4e532684de521b885f385d0841030efa2b1a20bafb76133a5e1379c1)
- [GPT4All embedding model](https://python.langchain.com/docs/integrations/text_embedding/gpt4all)

# 1. Initial Setup
This setup includes loading environment variables from a `.env` file, setting the required environment variables, and importing the necessary modules for further processing. It ensures that the code has access to the required APIs and functions for the subsequent tasks.


In [1]:
# Initial set up
from dotenv import load_dotenv
import os

# Load the variables from .env file and set the API key (or user may manually set the API key)
load_dotenv()  # This loads the variables from .env (not part of repo)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["LANGCHAIN_TRACING_V2"] = "true" # use when you want to debug or monitor the performance of your langchain applications
#os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY') # use when accessing cloud-based language models or services that langchain integrates with

# Import the necessary modules
import bs4
from langchain import hub
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel # for RAG with source
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from IPython.display import display, Markdown, Latex
import glob
import chromadb

In [2]:
# use_OpenAI
embeddings_model = OpenAIEmbeddings()
db_directory = "../data/chroma_db1"
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", 
                 temperature=0) # context window size 16k for GPT 3.5 Turbo

# Open source models used here are for illustration and educational purposes
# Added _ALT at the end of variables to indicate alternative method as a comparison
embeddings_model_ALT = GPT4AllEmbeddings()
db_directory_ALT = "../data/chroma_db2"
# define a local large language model for the augmented generation
# Ollama is one way to easily run inference
#llm = Ollama(model="solar:10.7b-instruct-v1-q5_K_M")
llm_ALT = Ollama(model="mistral:instruct")

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


# 2. Load PDF Files and Convert to a Vector DB
1. Create a function to load and extract text from PDF files in a specified folder. It defines a function called `load_pdfs_from_folder()` that takes a folder path as input and returns a list of extracted text documents from the PDF files in that folder.

2. In the example, the folder path `../data/ASOP` is used, but you can modify it to point to your desired folder.

3. By calling the `load_pdfs_from_folder()` function with the folder path, the code loads the PDF files, extracts the text using the PyPDFLoader, and stores the extracted text documents in the `docs` list.

4. After loading and extracting the text, a `RecursiveCharacterTextSplitter` object is created with specific parameters for chunking the documents. The `split_documents()` method is then used to split the documents into smaller chunks based on the specified parameters.

5. Finally, a Chroma vectorstore is created from the document splits. The vectorstore uses the defined embedding model for embedding the chunks and is saved to the predefined directory.

In [3]:
# Run only when the DB directory is empty
if not os.path.exists(db_directory) or not os.listdir(db_directory):
    # Define a function to load and extract text from PDFs in a folder
    def load_pdfs_from_folder(folder_path):
        # Get a list of PDF files in the specified folder
        pdf_files = glob.glob(f"{folder_path}/*.pdf")
        docs = []
        for pdf_file in pdf_files:
            # Load the PDF file using the PyPDFLoader
            loader = PyPDFLoader(pdf_file) 
            # Extract the text from the PDF and add it to the docs list
            docs.extend(loader.load())
        return docs
    
    # Example folder path
    folder_path = '../data/ASOP'
    
    # Call the function to load and extract text from PDFs in the specified folder
    docs = load_pdfs_from_folder(folder_path)
    
    # Create a text splitter object with specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=200,
        length_function=len,)
    
    # Split the documents into chunks using the text splitter
    splits = text_splitter.split_documents(docs)
    
    # Create a Chroma vector database from the document splits, using OpenAIEmbeddings for embedding
    vectorstore = Chroma.from_documents(documents=splits, 
                                        embedding=embeddings_model, 
                                        persist_directory=db_directory)

# 3. Retrieve from the Vector DB 

In [4]:
# Get a Chroma vector database with specified parameters
vectorstore = Chroma(embedding_function=embeddings_model, 
                     persist_directory=db_directory)

## Alternative method using local open-source LLM
vectorstore_ALT = Chroma(embedding_function=embeddings_model_ALT, 
                     persist_directory=db_directory_ALT)


In [14]:
## Retrieve and RAG chain

# Create a retriever using the vector database as the search source
retriever = vectorstore.as_retriever(search_type="mmr", 
                                     search_kwargs={'k': 6, 'lambda_mult': 0.4})

## Alternative method using local open-source LLM
retriever_ALT = vectorstore_ALT.as_retriever(search_type="mmr", 
                                     search_kwargs={'k': 6, 'lambda_mult': 0.4})

# Use MMR (Maximum Marginal Relevance) to find a set of documents that are both similar to the input query and diverse among themselves
# Increase the number of documents to get, and increase diversity (lambda mult 0.5 being default, 0 being the most diverse, 1 being the least)

# Load the RAG (Retrieval-Augmented Generation) prompt
prompt = hub.pull("rlm/rag-prompt")

# Define a function to format the documents with their sources and pages
def format_docs_with_sources(docs):
    formatted_docs = "\n\n".join(doc.page_content for doc in docs)
    sources_pages = "\n".join(f"{doc.metadata['source']} (Page {doc.metadata['page'] + 1})" for doc in docs)
    # Added 1 to the page number assuming 'page' starts at 0 and we want to present it in a user-friendly way

    return f"Documents:\n{formatted_docs}\n\nSources and Pages:\n{sources_pages}"

# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

## Alternative method using local open-source LLM
# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs_ALT = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm_ALT
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source_ALT = RunnableParallel(
    {"context": retriever_ALT, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs_ALT)


# 4. Generate Q&A Functions

In [15]:
def generate_output():
    # Prompt the user for a question on ASOP
    usr_input = input("What is your question on ASOP?: ")

    # Invoke the RAG chain with the user input as the question
    output = rag_chain_with_source.invoke(usr_input)
    output_ALT = rag_chain_with_source_ALT.invoke(usr_input)

    # Generate the Markdown output with the question, answer, and context
    markdown_output = "### Question\n{}\n\n### Open AI Answer\n{}\n\n".format(output['question'], output['answer'])
    markdown_output += "### Local LLM Answer\n{}\n\n### Open AI Context\n".format(output_ALT['answer'])

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            markdown_output += "- **Open AI Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content

    markdown_output += "\n\n### Local LLM Context\n"

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output_ALT['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            markdown_output += "- **Local LLM Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content

    
    # Display the Markdown output
    display(Markdown(markdown_output))

# Example questions related to ASOPs
- explain ASOP No. 14
- What are the considerations in choosing what methods to use for asset adequacy testing?
- How are expenses reflected in cash flow testing based on ASOP No. 22?
- What is catastrophe risk?
- When do I update assumptions?
- What should I do when I do not have credible data to develop non-economic assumptions?

In [17]:
generate_output()

What is your question on ASOP?:  How do I consider what methods to use for asset adequacy testing?


### Question
How do I consider what methods to use for asset adequacy testing?

### Open AI Answer
When considering methods for asset adequacy testing, it is important to use appropriate analysis methods and professional judgment to determine the reasonableness of results. Techniques like cash flow testing and loss-ratio methods may be suitable depending on the nature of the asset, policy, and liability cash flows. The actuary should also consider quantifying the impacts of changes and select appropriate assumptions for the analysis.

### Local LLM Answer
 To consider methods for asset adequacy testing, refer to the acceptable testing methods mentioned in the context. These include a gross premium reserve test and analyzing assets, their appropriateness for the analysis method, and the information and analysis used to support the determination that the method is appropriate. Use professional judgment when choosing a testing method and determining which assumptions should be varied. (Refer to sections 3.1 and 3.3.2 in your document.)

### Open AI Context
- **Open AI Source 1**: ../data/ASOP/asop022_167.pdf, page 13:

3.4 Forming an Opinion with Respect to Asset Adequacy Analysis  
⎯The actuary should use   
appropriate analysis methods when forming an  opinion with respect to asset adequacy. In   
judging whether the results from the asset adequacy analysis are satisfactory, the actuary should use professional judgment in determ ining which of the following, or other,   
considerations apply:   
   
3.4.1 Reasonableness of Results  
⎯The actuary should review the modeled future economic   
and experience conditions and test results for reasonableness.
- **Open AI Source 2**: ../data/ASOP/asop052_189.pdf, page 30:

ASOP No. 52—September 2017    
    
25  (collected under the general heading of “asset ad equacy analysis”) in testing for adequacy of   
reserves in light of th e assets supporting them. Foremost among these techniques was cash flow   
testing. Asset adequacy analysis was designed as  an aggregate test to determine whether the   
insurer should establish reserves in excess of the statutory minimums and includes methods of   
quantifying this amount. To a degree, these same t echniques are paralleled in the determination   
of certain components of a principle-based valuation.   
 Product design features introduced since the 1980s have led to a need for additional guidance   
on how to reserve for products. Model Regulation 830, Valuation of Life Insurance Policies   
Model Regulation (XXX), and Actuarial Guideline 38 (AG 38),  Application of the Valuation of   
Life Insurance Policies Model Regulation (AXXX) , were developed to address concerns for
- **Open AI Source 3**: ../data/ASOP/asop022_203.pdf, page 22:

The reviewers believe the guidance is appropriate and therefore made no change in response to   
this comment.     
Comment    
   
   
Response  One commentator suggested clarifying that asset adequacy reserves established in prior years   
should be excluded when performing asset adequacy analysis.    
   
The reviewers believe the guidance is appropriate and therefore made no change in response to   
this comment.     
Comment    
   
   
Response  One commentator suggested modifying the language to remove the implication that asset   
adequacy analysis is a guarantee.    
   
The reviewers agree and modified the language.    
Section 3.1 .1, Analysis Methods    
Comment    
   
   
Response  One commentator proposed additional disclosure when liability cash flows have a material   
dependency on the asset cash flows and cash flow testing is not used.    
   
The reviewers believe the guidance covers these issues at the appropriate level of detail and made   
no change in response to this comment.    
Comment
- **Open AI Source 4**: ../data/ASOP/asop022_203.pdf, page 13:

should consider quantifying the impacts of these changes.       
   
The use of new methods, models, or assumptions for new liability  segments (for   
example, a new line of business or product) or new asset  amounts is not a change   
within the meaning of this section.   
   
3.1.11 Completeness When performing t he asset  adequacy analysis , the actuary  should   
take into account anticipated material cash flows  such as renewal premiums,   
guaranteed and nonguarant eed benefits  and charges , expenses, and taxes. In   
determining the asset s supporting the tested  reserves and other liabilities , the   
actuary should take into account any asset  segmentation system used by the   
company.
- **Open AI Source 5**: ../data/ASOP/asop022_167.pdf, page 12:

small number of large individual claims over a short-term period.   
   
e. Loss-ratio methods may be appropria te when the asset, policy, and other   
liability cash flows are of short dura tion. Under this method, moderately   
adverse deviations in the actuarial assumptions underlying the morbidity or mortality costs may be tested. Loss-ra tio methods are described in ASOP    
No. 5, Incurred Health and Disability Claims .   
   
 If the actuary is uncertain as to whet her moderately adverse deviations in the   
investment rate-of-return assumptions will have a material impact on the asset adequacy analysis results, then the actua ry should also test moderately adverse   
deviations in the investment rate-of-return risk assumptions.   The actuary should document the asset adequacy analysis methods chosen.    
   
3.3.3 Assumptions  
⎯In addition to selecting an appropriate analysis method, the actuary   
should select appropriate assumptions. Accepted methods include the following:
- **Open AI Source 6**: ../data/ASOP/asop044_160.pdf, page 14:

ASOP No. 44—September 2009    
   
   
7   
  selected for a particular purpose, at each subsequent measurement date, the actuary   
should consider whether the selected asset va luation method continue s to be appropriate   
for that purpose. The actuary is not requir ed to do a complete reassessment at each   
measurement date. However, if a significant ch ange in the principal’s objectives has been   
communicated to the actuary (see secti on 3.2.2), the actuary should review the   
appropriateness of the asset valuation method. Furthermore, if the asset valuation method   
is other than market value, the actuary s hould review the appropriateness of the asset   
valuation method if an event such as the following has occurred:   a. a significant change in the plan provis ions affecting cash flow (such as adding a   
lump sum payment option, or freezing or te rminating the plan), in the actuarial   
cost method or funding policy, or in participant demographics;


### Local LLM Context
- **Local LLM Source 1**: ../data/ASOP/asop022_167.pdf, page 11:

assets, policies, or other liabilities may var y, or where the present value of combined   
asset, liability, or other cash flows may vary  under different economic or interest-rate   
scenarios.    
  Asset adequacy analysis test methods  other than cash flow testing may be   
appropriate in other situations. The follo wing are examples of acceptable methods.   
These methods would test moderately adverse deviations in the actuarial   
assumptions, except for the investment ra te-of-return assumptions. The actuary   
should use professional judgment in choosing an appropriate testing method and in determining which assumptions should be varied for the particular test.   
   
a. A gross premium reserve test may be  appropriate where the policy and other   
liability cash flows are sensitive to mode rately adverse deviations in the   
actuarial assumptions underlying these cash flows. For example, this type of   
method may be appropriate for term insurance backed by noncallable bonds,
- **Local LLM Source 2**: ../data/ASOP/asop022_203.pdf, page 15:

analysis  (see section 3.1 );   
   
c. the assets chosen, the methodology used for their selection, and their   
appropriateness for the analysis method (see section 3.1);    
   
d. the asset adequacy analysis  methods chosen, and the information and analysis   
used to support the determination that the method is appropriate for the reserves   
and other liabilities  being tested (see section 3.1.1) ;
- **Local LLM Source 3**: ../data/ASOP/asop032_196.pdf, page 22:

Current Practices      
    
Tests of Financial Adequacy     
    
Several well -established formal methods are currently being used to test the financial adequacy of   
Social Insurance Programs, as well as measures developed to assess the actuarial status and   
sustainability of these Programs over different time periods.    
    
The frequency with which Programs assess their financial status varies. Some (OASDI, Medicare,   
and PBGC, for instance) evaluate their financial position each year, while others, such as the   
Railroad Retirement Board, may perform a valuation every third yea r.
- **Local LLM Source 4**: ../data/ASOP/asop022_203.pdf, page 1:

3.1.3 Reinsurance Ceded  5   
3.1.4 Aggregation During Testing 5   
3.1.5 Use of Cash Flows from Other Financial Calculations  5   
3.1.6 Separate Account Assets  6   
3.1.7 Management Action  6   
3.1.8 Use of Data or Analyses Predating the Valuation Date  7   
3.1.9 Testing Horizon  7   
3.1.10 Changes in Methods, Models, or Assumptions  7   
3.1.11 Completeness  7   
3.1.12 Reliance on Others for Data, Projections, and Supporting Analysis  8   
3.1.13 Subsequent Events  8   
3.2 Forming an Opinion with Respect to Asset Adequacy Analysis  8   
3.2.1 Reasonableness of Results  8   
3.2.2 Adequacy of Reserves and Other Liabilities  8
- **Local LLM Source 5**: ../data/ASOP/asop022_167.pdf, page 1:

Section 3.  Analysis of Issues and Recommended Practices 4   
3.1 Requirements to Consider 4 3.2 Appointed or Qualified Actuary 4 3.3 Statement of Opinion 4   
3.3.1 Asset Adequacy Analysis 4 3.3.2 Analysis Methods 5 3.3.3 Assumptions 6 3.3.4 Additional Considerations 7   
3.4 Forming an Opinion with Respect to Asset Adequacy Analysis 7   
3.4.1 Reasonableness of Results 7
- **Local LLM Source 6**: ../data/ASOP/asop007_128.pdf, page 2:

3.2.1 Reasons for Cash Flow Testing 4 3.2.2 Cash Flow Testing is Not Always Necessary 4 3.2.3 Use of Analyses or Data Predating the Analysis Date 5   
3.3 Identification of Assets 5   
3.3.1 Choice of Asset Subsets to Use 5 3.3.2 Notional Asset Portfolios 5 3.3.3 Other Assets 5   
3.4 Projection of Asset Cash Flows 5   
3.4.1 Asset Characteristics 6 3.4.2 Investment Strategy 6


# 5. References
- https://www.actuarialstandardsboard.org/standards-of-practice/
- https://python.langchain.com/docs/use_cases/question_answering/quickstart
- https://python.langchain.com/docs/use_cases/question_answering/sources
- https://python.langchain.com/docs/integrations/text_embedding/
- https://docs.gpt4all.io/gpt4all_python_embedding.html#gpt4all.gpt4all.Embed4All
- https://chat.langchain.com/